In [1]:
import causalBootstrapping as cb

ModuleNotFoundError: No module named 'causalBootstrapping'

In [4]:
from distEst_lib import MultivarContiDistributionEstimator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.metrics import classification_report

In [5]:
causal_graph = '"Alcohol Consumption Causal Graph"; \
                D; F; S; A; P; M; N; J; K; \
                F -> D; \
                S -> D; \
                A -> D; \
                P -> D; \
                M -> D; \
                N -> D; \
                J -> D; \
                K -> D; \
                S -> F; \
                A -> F; \
                P -> F; \
                M -> F; \
                N -> F; \
                J -> F; \
                K -> F; \
                M -> J; \
                N -> K; '
# Dalc(D); Famrel(F); Sex(S); Age(A); Pstatus(P); Medu(M); Fedu(N); Mjob(J); Fjob(K)

In [7]:
weight_func_lam, weight_func_str = cb.general_cb_analysis(causal_graph = causal_graph, 
                                                          effect_var_name = 'D', 
                                                          cause_var_name = 'S',
                                                          info_print = True,
                                                          idmode = "shortest", 
                                                          idgreedy = True)

Interventional prob.:p_{S}(D)=\sum_{A,K,P,M,N,J}[p(S,A,P,D,M,J|K,N)p(S,A,N,P,K|J,M)/p(S)p(S)p(A)p(P)]
Causal bootstrapping weights function: [P(A,J,K,M,N,P,S)K(S,S')]/N*[P(S)P(S)P(A)P(P)P(K,N)P(J,M)]
Required distributions:
1: P(A,J,K,M,N,P,S)
2: P(S)
3: P(S)
4: P(A)
5: P(P)
6: P(K,N)
7: P(J,M)
Kernel function required: K(S,S')


In [90]:
from sklearn.preprocessing import OrdinalEncoder
import os
from sklearn.model_selection import train_test_split

# Load the dataset
total_data = pd.read_csv("./student-encoded.csv")
total_data.drop(labels = ['goout', 'health'], axis=1)
# Specify the desired features and target column
desired_features = ['sex', 'age', 'Pstatus','Medu','Fedu','famrel' ,'Mjob', 'Fjob', 'Dalc','Walc']
df = total_data[desired_features].copy()

df = pd.get_dummies(df, columns=['Mjob', 'Fjob'])

target_column = df['Dalc']

In [91]:
df

,sex,age,Pstatus,Medu,Fedu,famrel,Dalc,Walc,Mjob_at_home,Mjob_health,Mjob_other,Mjob_services,Mjob_teacher,Fjob_at_home,Fjob_health,Fjob_other,Fjob_services,Fjob_teacher
0,0.0,18,0.0,4,4,4,1,1,True,False,False,False,False,False,False,False,False,True
1,0.0,17,1.0,1,1,5,1,1,True,False,False,False,False,False,False,True,False,False
2,0.0,15,1.0,1,1,4,2,3,True,False,False,False,False,False,False,True,False,False
3,0.0,15,1.0,4,2,3,1,1,False,True,False,False,False,False,False,False,True,False
4,0.0,16,1.0,3,3,4,1,2,False,False,True,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1039,0.0,19,1.0,2,3,5,1,2,False,False,False,True,False,False,False,True,False,False
1040,0.0,18,1.0,3,1,4,1,1,False,False,False,False,True,False,False,False,True,False
1041,0.0,18,1.0,1,1,1,1,1,False,False,True,False,False,False,False,True,False,False
1042,1.0,17,1.0,3,1,2,3,4,False,False,False,True,False,False,False,False,True,False


In [92]:
# Create a file to save the split data
output_dir = "./split_data/"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Iterate over each feature and split the dataset
for column in df.columns:
    # Get the data for the current feature
    feature_data = df[[column]]
    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(feature_data, target_column, test_size=0.2, random_state=42)
    
    # Save the split training and testing sets as CSV files
    train_filename = os.path.join(output_dir, f"{column}_train.csv")
    test_filename = os.path.join(output_dir, f"{column}_test.csv")
    X_train.to_csv(train_filename, index=False)
    X_test.to_csv(test_filename, index=False)
    
    # Print the sizes of training and testing sets for each feature
    print(f"Feature: {column}")
    print(f"Training set size: {len(X_train)}")
    print(f"Testing set size: {len(X_test)}")
    print("-----------")

Feature: sex
Training set size: 835
Testing set size: 209
-----------
Feature: age
Training set size: 835
Testing set size: 209
-----------
Feature: Pstatus
Training set size: 835
Testing set size: 209
-----------
Feature: Medu
Training set size: 835
Testing set size: 209
-----------
Feature: Fedu
Training set size: 835
Testing set size: 209
-----------
Feature: famrel
Training set size: 835
Testing set size: 209
-----------
Feature: Dalc
Training set size: 835
Testing set size: 209
-----------
Feature: Walc
Training set size: 835
Testing set size: 209
-----------
Feature: Mjob_at_home
Training set size: 835
Testing set size: 209
-----------
Feature: Mjob_health
Training set size: 835
Testing set size: 209
-----------
Feature: Mjob_other
Training set size: 835
Testing set size: 209
-----------
Feature: Mjob_services
Training set size: 835
Testing set size: 209
-----------
Feature: Mjob_teacher
Training set size: 835
Testing set size: 209
-----------
Feature: Fjob_at_home
Training set s

In [93]:
total_data = df
total_data[['Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher']]

,Mjob_at_home,Mjob_health,Mjob_other,Mjob_services,Mjob_teacher
0,True,False,False,False,False
1,True,False,False,False,False
2,True,False,False,False,False
3,False,True,False,False,False
4,False,False,True,False,False
...,...,...,...,...,...
1039,False,False,False,True,False
1040,False,False,False,False,True
1041,False,False,True,False,False
1042,False,False,False,True,False


In [143]:
# Dalc(D); Famrel(F); Sex(S); Age(A); Pstatus(P); Medu(M); Fedu(N); Mjob(J); Fjob(K)
# Read demo data
testdata_dir = "./split_data/"
D_train = pd.read_csv(testdata_dir + "Dalc_train.csv")
F_train = pd.read_csv(testdata_dir + "famrel_train.csv")
S_train = pd.read_csv(testdata_dir + "sex_train.csv")
A_train = pd.read_csv(testdata_dir + "age_train.csv")
P_train = pd.read_csv(testdata_dir + "Pstatus_train.csv")
M_train = pd.read_csv(testdata_dir + "Medu_train.csv")
N_train = pd.read_csv(testdata_dir + "Fedu_train.csv")
#J_train = pd.read_csv(testdata_dir + "Mjob_train.csv")
#K_train = pd.read_csv(testdata_dir + "Fjob_train.csv")
#D_train = total_data['Dalc']
#F_train = total_data['famrel']
#S_train = total_data['sex']
#A_train = total_data['age']
#P_train = total_data['Pstatus']
#M_train = total_data['Medu']
#N_train = total_data['Fedu']
J_train = total_data[['Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher']]
K_train = total_data[['Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher']]
'''
Jathome_train = total_data['Mjob_at_home']
Jhealth_train = total_data['Mjob_health']
Jother_train = total_data['Mjob_other']
Jservices_train = total_data['Mjob_services']
Jteacher_train = total_data['Mjob_teacher']
Kathome_train = total_data['Fjob_at_home']
Khealth_train = total_data['Fjob_health']
Kother_train = total_data['Fjob_other']
Kservices_train = total_data['Fjob_services']
Kteacher_train = total_data['Fjob_teacher']
'''

# Reform the data to the acceptable format for the causalbootstrapping interfaces
D_train = np.array(D_train)
F_train = np.array(F_train)
S_train = np.array(S_train)
A_train = np.array(A_train)
P_train = np.array(P_train)
M_train = np.array(M_train)
N_train = np.array(N_train)
J_train = np.array(J_train)
K_train = np.array(K_train)

# do we have to encode True False to binary values?
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
J_train = encoder.fit_transform(J_train)
K_train = encoder.fit_transform(K_train)
'''
Jathome_train = np.array(Jathome_train)
Jhealth_train = np.array(Jhealth_train)
Jother_train = np.array(Jother_train)
Jservices_train = np.array(Jservices_train)
Jteacher_train = np.array(Jteacher_train)
Kathome_train = np.array(Kathome_train)
Khealth_train = np.array(Khealth_train)
Kother_train = np.array(Kother_train)
Kservices_train = np.array(Kservices_train)
Kteacher_train = np.array(Kteacher_train)
'''
data = {"D'": D_train,
        "F": F_train,
        "S": S_train,
        "A": A_train,
        "P": P_train,
        "M": M_train,
        "N": N_train,
        "J": J_train,
        "K": K_train}


In [106]:
# Interventional prob.:p_{S}(D)=\sum_{N,P,K,A,J,M}[p(M,P,A,J,S,D|N,K)p(N,P,K,A,S|J,M)/p(S)p(P)p(S)p(A)]
# Causal bootstrapping weights function: [P(A,J,K,M,N,P,S)K(S,S')]/N*[P(S)P(P)P(K,N)P(S)P(A)P(J,M)]
# Required distributions:
# 1: P(A,J,K,M,N,P,S)
# 2: P(S)
# 3: P(P)
# 4: P(K,N)
# 5: P(S)
# 6: P(A)
# 7: P(J,M)
# Kernel function required: K(S,S')

In [174]:

# Set number of the bins for histogram becasue all variables follow discrete distributions.
n_bins_sfd = [3,0,0,0,0,0,0,0,0,0,0,0,0,0]
n_bins_sf = [0,0]

data_ajkmnp = np.concatenate((A_train, J_train, K_train, M_train, N_train, P_train), axis=1)
data_kn = np.concatenate((K_train, N_train), axis=1)
data_jm = np.concatenate((J_train, N_train))



data_sdf = np.concatenate((S_train, D_train, F_train), axis = 1)
data_sd = np.concatenate((S_train, D_train), axis = 1)

dist_estimator_sfd = MultivarContiDistributionEstimator(data_fit=data_sdf, n_bins = n_bins_sfd)
pdf_sfd, psfd = dist_estimator_sfd.fit_histogram()
dist_estimator_sf = MultivarContiDistributionEstimator(data_fit=data_sd, n_bins = n_bins_sf)
pdf_sf, psf = dist_estimator_sf.fit_histogram()


dist_map = {tuple(sorted(["A", "J", "K", "M", "N", "P"])): lambda 
            tuple(sorted(["S","D","F"])): lambda S, D, F: pdf_sfd([S, D, F]),
            tuple(sorted(["S","D'","F"])): lambda S,D_prime, F: pdf_sfd([S, D_prime, F]),
            tuple(sorted(["S","D'"])): lambda S, D_prime: pdf_sf([S,D_prime]),
            tuple(sorted(["S","D"])): lambda S, D: pdf_sf([S, D])}



In [179]:
psfd

array([[[0.01437126, 0.01556886, 0.06826347, 0.22155689, 0.11856287],
        [0.00239521, 0.00718563, 0.01916168, 0.04550898, 0.01197605],
        [0.0011976 , 0.00239521, 0.00239521, 0.00838323, 0.00359281],
        [0.00239521, 0.        , 0.00479042, 0.0011976 , 0.0011976 ],
        [0.        , 0.        , 0.        , 0.0011976 , 0.00239521]],

       [[0.        , 0.00838323, 0.03592814, 0.11377246, 0.09341317],
        [0.0011976 , 0.00598802, 0.0251497 , 0.05269461, 0.01676647],
        [0.00359281, 0.00479042, 0.00359281, 0.0239521 , 0.02035928],
        [0.        , 0.        , 0.00239521, 0.00838323, 0.00598802],
        [0.0011976 , 0.0011976 , 0.00239521, 0.00598802, 0.00718563]]])

In [176]:
dist_map.keys()

dict_keys([('D', 'F', 'S'), ("D'", 'F', 'S'), ("D'", 'S'), ('D', 'S')])

In [183]:
cb_data = cb.general_causal_bootstrapping_simple(weight_func_lam = weight_func_lam, 
                                                 dist_map = dist_map, data = data, 
                                                 intv_var_name = 'D', kernel = None)

KeyError: ('A', 'J', 'K', 'M', 'N', 'P', 'S')